<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_tower_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 設定に絞り込みたいeNB-LCIDを入力
2. 上部メニューより「ランタイム」を選択
3. 「すべてのセルを実行」を選択
4. アップロードで一時停止しているので参照ボタンを押してTowerCollectorのCSVファイルをアップロード
5. 最後にマップが表示される

In [ ]:
f = lambda enb, lcid: (enb << 8) | lcid

# 設定

In [ ]:
# eNB-LCID

cells = [f(344574, 1)]

# 複数の場合
# cells = [f(737430, 10), f(737430, 11), f(737430, 12)]

cells

# アップロード

参照ボタンを押してCSVファイルをアップロード

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# プログラム

In [ ]:
!pip install -U pandas

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

In [ ]:
import folium
from folium.features import DivIcon
from folium import plugins

In [ ]:
# マップ作成

def make_map(df, zoom=14):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=zoom)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
    ).add_to(map)

    # Googleマップ航空写真
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map(航空写真)",
        attr="<a href='https://developers.google.com/maps/documentation'>© Google</a>",
    ).add_to(map)

    fg1 = folium.FeatureGroup(name="Maker").add_to(map)
    fg2 = folium.FeatureGroup(name="TA").add_to(map)
    fg3 = folium.FeatureGroup(name="RSRP", show=False).add_to(map)

    for i, r in df.iterrows():

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=r.color,
            fill=True,
            tooltip=r["id"],
        ).add_to(fg1)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["measured_at"]}<br />eNB-LCID:{r["id"]}<br />PCI:{r["psc"]}<br />TA:{r["ta"]}<br />RSRP:{r["rsrp"]}',
        ).add_to(fg2)
    
        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(30, 20),
                icon_anchor=(15, -10),
                html=f'<div style="text-align:center; font-size: 9pt; font-weight: bold">{r["rsrp"]}</div>',
            ),
            tooltip=f'{r["measured_at"]}<br />eNB-LCID:{r["id"]}<br />PCI:{r["psc"]}<br />TA:{r["ta"]}<br />RSRP:{r["rsrp"]}',
        ).add_to(fg3)

    folium.LayerControl().add_to(map)

    # DRAW
    folium.plugins.Draw(
        draw_options={"polygon": False, "rectangle": False, "circlemarker": False}
    ).add_to(map)

    return map

In [ ]:
# ヒートマップ作成

def make_heatmap(df, zoom=14):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=zoom)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>©Google Map</a>",
    ).add_to(map)

    # Googleマップ航空写真
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map(航空写真)",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>© Google</a>",
    ).add_to(map)

    folium.plugins.HeatMap(
        data = df.values,
        radius=10,    
    ).add_to(map)

    return map

# データラングリング

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"]).dropna(how="all", axis=1)
df0

In [ ]:
df0.dtypes

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11 & net_type == 'LTE'").copy()
df1

In [ ]:
# 日時変換
df1["measured_at"] = df1["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [ ]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [ ]:
df1["ta"] = df1["ta"].astype("Int64")
df1["rsrp"] = df1["rsrp"].astype(int)

In [ ]:
# 色分け（青 > 緑 > 黄 > 橙 > 赤）

df1["color"] = pd.cut(
    df1["rsrp"],
    [-140, -100, -90, -80, -40],
    labels=["red", "orange", "yellow", "green"],
).astype(str)

idx = df1.groupby("cell_id")["rsrp"].transform(max) == df1["rsrp"]
df1.loc[idx, "color"] = "blue"

# 地図

## データ全部

In [ ]:
# 地図表示
map1 = make_map(df1)
map1

In [ ]:
map1.save("map1.html")

## 絞り込み

In [ ]:
# 絞り込み
df2 = df1[df1["cell_id"].isin(cells)].copy()

# データ確認
df2[["measured_at", "id", "rsrp", "ta", "lat", "lon"]]

In [ ]:
# 地図表示
map2 = make_map(df2, 16)
map2

In [ ]:
map2.save("map2.html")

## ヒートマップ

In [ ]:
# RSRPのしきい値
N = 100

In [ ]:
df3 = df2.copy()

In [ ]:
# しきい値を加算
df3["weight"] = df3["rsrp"] + N

In [ ]:
df4 = df3.reindex(columns=["lat", "lon", "weight"]).copy()

In [ ]:
# 0以下は除外のため確認
df4["weight"].value_counts().plot.bar()

In [ ]:
# 地図表示
map3 = make_heatmap(df4[df4["weight"] > 0])
map3

In [ ]:
map3.save("map3.html")

## ダウンロード

In [ ]:
# files.download("map1.html")
files.download("map2.html")
# files.download("map3.html")

## 時間帯指定

In [ ]:
# RSRPの時系列確認
df2.plot(x="measured_at", y="rsrp")

In [ ]:
# TAの時系列確認
df2.dropna(subset=["ta"]).plot(x="measured_at", y="ta")

In [ ]:
# 時間帯指定
TM_Start = "16:39:00"
TM_End = "16:41:00"

In [ ]:
df5 = df2.set_index("measured_at").between_time(TM_Start, TM_End).reset_index()
df5

In [ ]:
make_map(df5, 16)